### Imports

In [163]:
import gurobipy as gp
import pandas as pd
import os

### Functions

In [164]:
def extract(data, row, fact, maximize=True):
    text = str(data.iloc[row][fact]).strip("mg%?")
    if str(text) == "nan" or len(text) == 0:
        return maximize * 1_000_000
    return float(text)

In [165]:
def classify_age(a: int) -> str:
    a = round(a, 0)
    age_strings = ["1-3", "4-8", "9-13", "14-18", "19-30", "31-50", "51-110"]
    age_groups = [tuple([int(age) for age in i.split("-")]) for i in age_strings]
    for i, age in enumerate(age_groups):
        if age[0] <= a <= age[1]:
            return age_strings[i]
    else:
        raise ValueError(f"{a} is not a valid age")

In [166]:
def init_guide_amts(guide):
    print("Converting Percents to Amounts...")
    for col in guide.columns:
        if "% kcal" in col:
            print(col)
            guide[col[:-6] + "AMT"] = pd.Series([0] * len(guide.index))
            for row in range(len(guide.index)):
                cal_levels = [float(i) for i in str(guide.iloc[row]["Calorie Level(s)"]).replace(",", "").split("/")]
                avg_kcals = sum(cal_levels) / len(cal_levels) / 1000
                percents = [float(i) for i in str(guide.iloc[row][col]).strip("< %").replace(",", "").split("-")]
                nut_amt = avg_kcals * sum(percents) / len(percents)
                guide.loc[:, col[:-6] + "AMT"].iloc[row] = nut_amt
    return guide

### Restaurant / Category Selection

In [167]:
def filter_subset(df: pd.DataFrame, restaurant: str, categories: list=[]) -> pd.DataFrame:
    cats = []  # ["Breakfast", "Beverages"]
    subset = df
    subset = subset[subset["Restaurant"] == restaurant]
    if categories != []:
        subset = subset[subset["Common Category"].isin(categories)]
#     if real_meal and not cat_limit:
#         bad_cats = ["Drinks", "Beverages", "Sauces and Condiments", "Coffee", "Salad Dressing"
#                     "Ocean Water", "Iced Teas", "Limeades", "Soft Drinks", ]
#         subset = subset[~subset["Category"].isin(bad_cats)]
    subset.reset_index(inplace=True)
    return subset  # Contains all viable food entries

### File Input

In [168]:

def load_ref_files(nutrition_name="Nutritional Facts - Categorized", guidelines_name="Dietary Guidelines"):
    df = pd.read_excel(nutrition_name + ".xlsm", index_col=0)
    guide = pd.read_excel(guidelines_name + ".xlsx", header=1, index_col=[1, 2], skiprows=[2]).drop("Unnamed: 0", axis=1)
    guide = init_guide_amts(guide)
    return df, guide

### Requirement Selection

In [169]:
def get_requirements(subset, guide):
    filters = {key: value for key in subset.columns for value in guide.index if "%" not in key and key + ', mg' == value or key + ', g' == value}
    filters.update({"Total Carbohydrates": "Carbohydrate, g", "Sugars": "Added Sugars, AMT"})
    less_thans = ["Sodium", "Sugars", "Calories From Fat"]
    return filters, less_thans

In [170]:
def guide_lookup(gender: str, age: int, guide: pd.DataFrame):
    genders = {"m": "Male", "f": "Female"}
    gender = genders[gender[0].lower()]
    return guide.loc[gender, classify_age(age)]

### Model Building: Variables, Constraints, and Objective

In [187]:
def create_model(subset: pd.DataFrame, filters: dict, less_thans: list, guide: pd.DataFrame,
                 objective: str = "Calories From Fat", min_obj: bool = True, min_cal_cutoff: int = 1,
                 real_meal: bool = True, cat_limit: bool = True, meals: int = 2, verbose: bool = True,
                 filter_relaxations: list = []):
    """
    Generates the Gurobi model according to many available filters and options. Objective is the most changing default option.
    Other defaults are usually okay, with the possible exception of filter_relaxations, which handles infeasible models.
    
    :subset: menu items to consider, frequently broken down by restaurant
    :filters: dictionary of nutrition requirements (sugars, proteins, etc.). Inequalities may go either way (specified in less_thans)
    :less_thans: list of filters where the sum of food nutritions must be less than the guideline (sodium) rather than more (protein)
    :guide: dietary recommendations taken from the government's Dietary Guidelines For Americans. TODO: Update to 2020-2025 data
    :objective: dietary fact by which the model should be optimized. By default, this is minimized by min_obj
    :min_obj: whether the objective ought to [True] minimized (sodium, sugar, calories, etc.) or [False] maximized (protein, vitamins, ...)
    :min_cal_cutoff: when non-zero, omits foods from the solution with fewer calories than the cutoff. Enabled by default
    :real_meal: enforces the minimum calorie cutoff above. Primarily to omit condiments with 'free' nutrition
    :cat_limit: requires no more than cat_limit of any one food type in the solution. Prevents 31 apple juice box solutions
    :meals: requires the solution to meet only (1 / meals) of each nutrient recommendation. Meals=1 & cat_limit=True usually infeasible
    :verbose: provides detailed constraining / solving progress updates. Disable for more concise output
    :filter_relaxations: which requirements are allowed to be relaxed if an optimal solution is not otherwise possible. [Sugars, sodium, calories, ...]
    """
    if verbose:
        vprint = print
    else:
        vprint = lambda *x, **y: None
    m = gp.Model()
    xis = [m.addVar(vtype=gp.GRB.CONTINUOUS) for _ in subset.index]  # GRB.BINARY / GRB.INTEGER, whether to include a food in the meal
    f_rel = {fact: m.addVar(name=f"{fact}_rel") if fact in filter_relaxations else 0 for fact in filters}  # excess variable
    
    for fact, req in filters.items():  # Nutrition Requirements
        if fact in less_thans:
            vprint(f"Constraining {fact}".ljust(35), f"<= {round(guide[filters[fact]], 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) - f_rel[fact] <= float(guide[filters[fact]]) / meals)
        else:
            vprint(f"Constraining {fact}".ljust(35), f">= {round(guide[filters[fact]], 2)}".ljust(10), f"across {meals} meals")
            m.addConstr(sum((x * extract(subset, r, fact, maximize=False) for r, x in enumerate(xis))) >= float(guide[filters[fact]]) / meals)

    if real_meal:
        for i, food in enumerate(subset.iloc):  # Excludes zero calorie (from fat) entries
            m.addConstr(xis[i] <= extract(subset, i, "Calories From Fat", maximize=True) * min_cal_cutoff)
            # TODO: Evaluate whether the above line is correct. I think it's carryover from when min_cal_cutoff was a boolean
    if cat_limit:
        vprint()
        for cat in pd.unique(subset["Common Category"]):
            vprint(f"Constraining only {cat_limit} or fewer {cat.strip('s')} items.")
            m.addConstr(sum([x for i, x in enumerate(xis) if subset["Common Category"][i] == cat]) <= cat_limit)
    
    m.setParam("OutputFlag", verbose)
    m.ModelSense = gp.GRB.MINIMIZE if min_obj else gp.GRB.MAXIMIZE  # Less concise, more clear than '2 * min_obj - 1'
    costs = {"Sugars": 1, "Sodium": 50, "Calories From Fat": 4}  # Note units: Sugars (g) vs Sodium (mg)
    overage_cost = [f_rel[fact] * costs[fact] if fact in costs else f_rel[fact] for fact in filters]
    m.setObjective(sum((x * extract(subset, r, objective, maximize=True) for r, x in enumerate(xis))) + sum(overage_cost))
    return m

### Meal Output

In [183]:
def display_details(m, model_args, subset):
    best_choices = [i for i, x in enumerate(m.getVars()) if x.x > 0 and "rel" not in x.VarName]
#     print(sum([m.x[i] for i in best_choices]), "calories from fat")  # TODO: Appears to produce incorrect output...
    
#     print(subset.iloc[best_choices]["Food"], ":")
    selection = subset.iloc[best_choices]
    print(f"The following foods will satisfy your requirements across {model_args['meals']} meals, subject to these overages:", overages)
    display(selection)

### Solving

In [184]:
def solve_models(filter_relaxations: list, objective: str, feasibility: bool=True, full_run: bool=False, verbose: bool=True):
    """
    Attempts to find feasible solutions for all restaurants, objectives, and relaxations.
    :filter_relaxations: which constraints may be relaxed. Require an associated cost, set in create_model()
    :objective: which nutrient to rank by
    :feasibility: try looser constraints if the initial model is infeasible
    :full_run: supress interactive prompts and run full dataset
    :verbose: display operation messages. full_run will take several seconds
    """
    if verbose:
        vprint = print
    else:
        vprint = lambda *x, **y: None
    feasible = set()
    rel_feasible = set()
    weak_feasible = set()
    infeasible = set()
    for res in pd.unique(df["Restaurant"]):
        vprint()
        subset = filter_subset(df, res)
        filters, less_thans = get_requirements(subset, guide)
        model_args = {"subset": subset, "filters": filters, "less_thans": less_thans,
                      "objective": objective, "min_obj": True, "guide": guide,
                      "verbose": False, "filter_relaxations": filter_relaxations, "meals": 2,
                      "cat_limit": 200, "real_meal": False}
        m = create_model(**model_args)
        m.optimize()
        if m.status != 2:
            vprint(f"The strict {res} model is infeasible.")
            if feasibility:
                if full_run:
                    verbose_retry = "R"
                else:
                    verbose_retry = input(f"""'S'kip {res};
                                              'R'etry with loosened constraints (default);
                                              'V'erbose retry; [S/R/V] """).upper()
            else:
                continue
            if verbose_retry == "S":
                continue
            weak_model_args = {"subset": subset, "filters": filters, "less_thans": less_thans,
                               "objective": objective, "min_obj": True, "min_cal_cutoff": -1,
                               "guide": guide, "verbose": verbose_retry == "V",
                               "filter_relaxations": filter_relaxations, "meals": 2,
                               "real_meal": False, "cat_limit": 10}
            m = create_model(**weak_model_args)
            m.optimize()
            if m.status == 2:
                weak_feasible.add(res)
        if m.status == 2:
    #         objval = sum([x.x * subset.loc[:, "Calories From Fat"].iloc[i] if "rel" not in x.VarName else 0 for i, x in enumerate(m.getVars())])
            overages = [x for i, x in enumerate(m.getVars()) if x.x > 0 and "rel" in x.VarName]
            if sum([v.x for v in overages]) == 0:
                vprint(f"The {res} model is feasible:", "\t" * 5, "<" + "-" * 10)
                if res not in weak_feasible:
                    feasible.add(res)
            else:
                vprint(f"The {res} model is feasible with the following relaxations:\t\t", overages)
                if res not in weak_feasible:
                    rel_feasible.add(res)
        else:
            vprint(f"The {res} model is still infeasible. Skipping.")
            infeasible.add(res)
        if full_run:
            directions = "S"
        else:
            directions = input(f"'S'olve next restaurant (default); 'E'xit this loop; 'D'etails about current solution; [S/E/D] ").upper()
        if len(directions) == 0 or directions[0] == "S":
            continue
        elif directions[0] == "D":
            if m.status == 2:
                display_details(m, model_args, subset)
            else:
                vprint("The model is infeasible - no details are available")
        elif directions[0] == "E":
            break
    # print("Return reached...")        
    return {"Feasible": feasible, "Relaxed": rel_feasible, "Weakened": weak_feasible, "Infeasible": infeasible}

### Run Config: Open Datasources & Customize Evaluation

In [186]:
df, guide = load_ref_files()
age, gender = 25, "Male"
guide = guide_lookup(gender, age, guide)

attempt_feasibility = True       # Relax real_meal and cateogry limits
full_run = True                  # Disable interactive style to run through all models quickly
verbose = True                   # Solving, solution, and output solutions
out_name = "FeasibilityResults-InitiallyWeakened-ContinuousVars"  # Result output filename

types = ["feasible", "rel_feasible", "weak_feasible", "infeasible"]
results = pd.DataFrame(columns=["Sugars", "Sodium", "Calories From Fat"], index=pd.unique(df["Restaurant"]))

Converting Percents to Amounts...
Protein, % kcal
Carbohydrate, % kcal
Added Sugars, % kcal
Total Fat, % kcal
Saturated Fat, % kcal


### Partial Evaluation

In [188]:
for objective in ["Sugars", "Sodium", "Calories From Fat"]:
    if verbose:
        print(f"Ranking by {objective} and relaxing by {filter_relaxations}:")
    filter_relaxations = ["Sugars", "Sodium", "Calories From Fat"]
    del filter_relaxations[filter_relaxations.index(objective)]
    
    groups = solve_models(filter_relaxations, objective,
                          feasibility = attempt_feasibility,
                          full_run = full_run, verbose = verbose)
    for t, restaurants in groups.items():
        for restaurant in restaurants:
            results.loc[restaurant, objective] = t

    if verbose:
        print(f"\n\nRanking by {objective} and relaxing by {filter_relaxations}:")
        for t, res in groups.items():
            print(t, "\t:\t", res)

if verbose:
    print(f"\n\nWriting results to {out_name}.xlsx...")
if not os.path.exists(f"./{out_name}.xlsx"):
    results.to_excel(out_name + ".xlsx")
if verbose:
    print("Done!")

Ranking by Sugars and relaxing by ['Sugars', 'Sodium']:

The Arby's model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 1940.7442680776014)>]

The strict Baskin-Robbins model is infeasible.
The Baskin-Robbins model is still infeasible. Skipping.

The Blimpie model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 313.66666666666674)>]

The strict Boston Market model is infeasible.
The Boston Market model is still infeasible. Skipping.

The Buffalo Wild Wings model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 879.3688888888893)>]

The Burger King model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 1337.4666666666667)>]

The Carl's Jr model is feasible with the following relaxations:		 [<gurobi.Var Sodium_rel (value 932.5)>]

The Chipotle model is feasible: 					 <----------

The Culvers model is feasible: 					 <----------

The Dairy Queen model is feasible with the f

The Taco John's model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 34.95757575757576)>]

The Wendy's model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 29.152380952380952)>]

The Whataburger model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 101.86666666666666)>]

The Zaxby's model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 59.91237050578915)>]

The Chick-fil-A model is feasible with the following relaxations:		 [<gurobi.Var Sugars_rel (value 111.46666666666667)>]

The McDonald's model is feasible: 					 <----------


Ranking by Sodium and relaxing by ['Sugars', 'Calories From Fat']:
Feasible 	:	 {"Godfather's Pizza", 'Popeyes', "McDonald's", 'Subway', 'Sonic'}
Relaxed 	:	 {"Papa John's", 'Little Caesars', 'Culvers', 'Dunkin Donuts', 'Taco Bell', 'Smashburger', 'Quiznos', "Domino's Pizza", "Wendy's", 'Baskin-Robbins', 'KFC', "Arby's", 'Del Taco', 'Chipotle',

### Next Steps

* Add ability to introspect any particular model to analyze food choices and relaxations.
* Work towards general metric capable of ranking restaurants against one another
* Consider adding other relaxations
* Write all optimal diets out a spreadsheet
* Update guide to 2020-2025 recommendations
* Talk about appropriate Calorie / Sugar / Sodium Balancing
* Multicriteria optimization so that the sum of all three overages is minimized, rather than just one plus the overages of the other two
* Understand why various alternate objectives aren't all feasible / infeasible together
* Get list of nutrients by which we are actually constraining
* Simplify model to obtain more feasible solutions (ignore vitamins) (use protein, fiber, calories, iron, sugars, sodium, etc.)

In [189]:
guide

Calorie Level(s)         2,400/2,600/3,000
Protein, g                              56
Protein, % kcal                      10-35
Carbohydrate, g                        130
Carbohydrate, % kcal                 45-65
Dietary Fiber, g                        34
Added Sugars, % kcal                   0.1
Total Fat, % kcal                    20-35
Saturated Fat, % kcal                  0.1
Linoleic Acid, g                        17
Linolenic Acid, g                      1.6
Calcium, mg                           1000
Iron, mg                                 8
Magnesium, mg                          400
Phosphorus, mg                         700
Potassium, mg                         4700
Sodium, mg                            2300
Zinc, mg                                11
Copper, mcg                            900
Manganese, mg                            2
Selenium, mcg                           55
Vitamin A, mg RAE                      900
Vitamin E, mg AT                        15
Vitamin D, 